In [1]:
!pip install gurobipy

In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

In [3]:
nfl = pd.read_csv('nflratings.csv', header=None)
print(nfl)

      0   1   2   3   4
0     1  25  31  13  10
1     1   2  17  19   7
2     1  29  26  28   0
3     1  21  32  23  17
4     1   3  16  38  24
..   ..  ..  ..  ..  ..
251  17   5  20  23  10
252  17  11   6  23  37
253  17  22   6  37   0
254  17   8  15  23  17
255  17   1  12   7  33

[256 rows x 5 columns]


In [4]:
home = nfl[1].values -1
away = nfl[2].values -1
homescore = nfl[3].values
awayscore = nfl[4].values

In [5]:
spread = homescore - awayscore
initial_ratings = np.zeros(32)
initial_hfa = 0

In [6]:
def obj(params):
    ratings = params[:32]
    hfa = params[32]
    
    pred_spread = ratings[home] - ratings[away] + hfa
    
    errors = spread - pred_spread
    
    return np.sum(errors**2)

In [7]:
def con(params):
    ratings = params[:32]
    return np.mean(ratings) - 85

In [8]:
initial_params = np.concatenate([initial_ratings, [initial_hfa]])

constraints = [{'type': 'eq', 'fun': con}]
bounds = [(None, None)]*33

result = minimize(obj, initial_params, constraints=constraints, bounds=bounds)

In [9]:
final_ratings = result.x[:32]
final_hfa = result.x[32]

predicted_spreads = final_ratings[home] - final_ratings[away] + final_hfa
predicted_winners = np.sign(predicted_spreads)
actual_winners = np.sign(spread)
correct_predictions = np.sum(predicted_winners == actual_winners)

In [10]:
print(f"Home field advantage: {final_hfa:.2f}")
print(f"Number games correctly predicted: {correct_predictions} out of 256")
print(f"Number games incorrectly predicted: {256-correct_predictions} out of 256")

Home field advantage: 2.17
Number games correctly predicted: 181 out of 256
Number games incorrectly predicted: 75 out of 256
